Import libraries


In [2]:
#import libraries
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import dataset

In [6]:
data = pd.read_csv('./drive/MyDrive/colab notebook/2 Sentiment Analysis/review-old-dump.csv')

In [7]:
data.shape

(197, 4)

In [8]:
data.head

<bound method NDFrame.head of       ProductId          UserId  Score  \
0    B001E4KFG0  A3SGXH7AUHU8GW      5   
1    B00813GRG4  A1D87F6ZCVE5NK      1   
2    B000LQOCH0   ABXLMWJIXXAIN      4   
3    B000UA0QIQ  A395BORC6FGVXV      2   
4    B006K2ZZ7K  A1UQRSCLF8GW1T      5   
..          ...             ...    ...   
192  B0028C44Z0  A19NGYCQ1NCF3W      5   
193  B0028C44Z0  A32DDKLY942A84      4   
194  B0028C44Z0  A3B6S35IUGOGW4      5   
195  B0028C44Z0   AI7O5C9KBVSTF      5   
196  B0028C44Z0   A4LL4KXLRBQHQ      3   

                                                  Text  
0    I have bought several of the Vitality canned d...  
1    Product arrived labeled as Jumbo Salted Peanut...  
2    This is a confection that has been around a fe...  
3    If you are looking for the secret ingredient i...  
4    Great taffy at a great price. There was a wide...  
..                                                 ...  
192  These little guys are tasty and refreshing.\nI...  
193  I li

Data Cleaning

In [9]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
import re

corpus = []
for review in data['Text']:
    review = re.sub('[^a-zA-Z]', ' ', review)  # Remove non-alphabetic characters
    review = review.lower()  # Convert text to lowercase
    review = review.split()  # Split text into individual words
    review = [ps.stem(word) for word in review if word not in set(all_stopwords)]  # Perform word stemming and remove stopwords
    review = ' '.join(review)  # Join processed words back into a single string
    corpus.append(review)  # Add processed review to the corpus

In [11]:
corpus

['bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better',
 'product arriv label jumbo salt peanut peanut actual small size unsalt not sure error vendor intend repres product jumbo',
 'confect around centuri light pillowi citru gelatin nut case filbert cut tini squar liber coat powder sugar tini mouth heaven not chewi flavor highli recommend yummi treat familiar stori c lewi lion witch wardrob treat seduc edmund sell brother sister witch',
 'look secret ingredi robitussin believ found got addit root beer extract order good made cherri soda flavor medicin',
 'great taffi great price wide assort yummi taffi deliveri quick taffi lover deal',
 'got wild hair taffi order five pound bag taffi enjoy mani flavor watermelon root beer melon peppermint grape etc complaint bit much red black licoric flavor piec not particular favorit kid husband last two week would recommend brand taffi delight treat',
 's

Data Transformation


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of CountVectorizer
cv = CountVectorizer()

# Transform the corpus into a matrix of token counts
X = cv.fit_transform(corpus).toarray()

# Extract the target variable values
y = data.iloc[:, -2].values

In [14]:
#Saving BoW dictionary to later use in prediction
import pickle
bow_path = './drive/MyDrive/colab notebook/2 Sentiment Analysis/ 3.1 BoW Sentiment Model.pk1'
pickle.dump(cv, open(bow_path, 'wb'))

Dividing dataset into training and testing


In [15]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Model Fitting(Naive Based)

In [16]:

from sklearn.naive_bayes import GaussianNB

# Create an instance of the Naive Bayes classifier
classifier = GaussianNB()
classifier.fit(X_train, y_train)



GaussianNB()

Export NB Classifier for later use

In [17]:
import joblib

# Specify the file path for saving the classifier
file_path = './drive/MyDrive/colab notebook/2 Sentiment Analysis/3.2 Classifier Sentiment Model.pkl'

# Save the classifier to the specified file path
joblib.dump(classifier, file_path)



['./drive/MyDrive/colab notebook/2 Sentiment Analysis/3.2 Classifier Sentiment Model.pkl']

Model Perfomance

In [19]:

y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cn = confusion_matrix(y_test, y_pred)
print(cn)
accuracy_score(y_test, y_pred)



[[ 0  0  1  0  2]
 [ 0  0  0  0  1]
 [ 0  0  0  0  1]
 [ 0  0  0  0  6]
 [ 0  0  0  0 29]]


0.725